In [37]:
import tensorflow as tf
from keras import layers, models
from keras.layers import Normalization, Lambda
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from scipy.ndimage import gaussian_filter1d
from sklearn.model_selection import train_test_split

In [38]:
def create_global_view():
    inputs = layers.Input(shape=(None, 3))  # Assuming the logs are 1D sequences
    
    x = layers.Masking(mask_value=-99)(inputs)  # Using -99 as the mask value

    # Encoding layer 1
    conv1 = layers.Conv1D(64, 3, activation="relu", padding="same")(x)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Dropout(0.5)(conv1)
    pool1 = layers.MaxPooling1D()(conv1)

    # Encoding layer 2
    conv2 = layers.Conv1D(128, 3, activation="relu", padding="same")(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Dropout(0.5)(conv2)
    pool2 = layers.MaxPooling1D()(conv2)

    # Middle layer
    conv_middle = layers.Conv1D(256, 3, activation="relu", padding="same")(pool2)
    conv_middle = layers.BatchNormalization()(conv_middle)
    conv_middle = layers.Dropout(0.5)(conv_middle)

    # Decoding layer 1
    up1 = layers.UpSampling1D()(conv_middle)
    
    # # Ensure sizes match before concatenating
    # cropped_conv2 = layers.Cropping1D(cropping=((1,0)))(conv2)
    merge1 = layers.concatenate([conv2, up1])
    
    decode1 = layers.Conv1D(128, 3, activation="relu", padding="same")(merge1)
    decode1 = layers.BatchNormalization()(decode1)
    decode1 = layers.Dropout(0.5)(decode1)

    # Decoding layer 2
    up2 = layers.UpSampling1D()(decode1)
    
    # Ensure sizes match before concatenating for the second merge
    # cropped_conv1 = layers.Cropping1D(cropping=((2,0)))(conv1)  # Adjusted cropping
    merge2 = layers.concatenate([conv1, up2])

    decode2 = layers.Conv1D(64, 3, activation="relu", padding="same")(merge2)
    decode2 = layers.BatchNormalization()(decode2)
    decode2 = layers.Dropout(0.5)(decode2)

    return models.Model(inputs, decode2)


In [39]:
# Local View
def create_local_view():
    inputs = layers.Input(shape=(None, 3))
    x = layers.Masking(mask_value=-99)(inputs)  # Using -99 as the mask values
    
    # Inception layers with dilated convolutions
    conv1 = layers.Conv1D(32, 1, padding="same", dilation_rate=1)(x)
    conv3 = layers.Conv1D(32, 3, padding="same", dilation_rate=2)(x)
    concat = layers.Concatenate()([conv1, conv3])
    return models.Model(inputs, concat)

In [40]:
# Normalize function
def normalize_data(data, mean, std):
    return (data - mean) / std


def ensure_even_length(sequence, padding_value=-99):
    # If the sequence length is odd, append padding to make it even
    if sequence.shape[0] % 2 != 0:
        padding_shape = list(sequence.shape)
        padding_shape[0] = 1  # we only need to add one row
        
        # Create a padding with the same number of columns as the sequence
        padding = np.full(padding_shape, padding_value)
        
        # Append padding to the sequence
        sequence = np.vstack([sequence, padding])
        
    return sequence

def pad_to_max_length(sequence, max_length, padding_value=-99):
    padding = [(0, max_length - sequence.shape[0]), (0, 0)]
    return np.pad(sequence, padding, mode='constant', constant_values=padding_value)


# Define a Gaussian smoothening function
def smooth_labels(y, sigma=3):
    return gaussian_filter1d(y, sigma)


In [41]:
current_dir = os.getcwd()
# Train your model here
folder_path = current_dir + "/Data/Well Log Data/"
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

dfs = []
for file in files:
    df = pd.read_csv(folder_path + file)
    dfs.append(df)

train_dfs, val_dfs = train_test_split(dfs, test_size=0.2, random_state=42) # Here 20% of the data is kept for validation

features = ["GR", "ILD", "DPHI"]

# Compute the mean and standard deviation of the training data
all_train_data = np.vstack([df[features].values for df in train_dfs])
mean = np.mean(all_train_data, axis=0)
std = np.std(all_train_data, axis=0)

# Extract features and labels for training and validation sets
X_train = np.array([df[features].values for df in train_dfs])
y_train = np.array([df["Pick"].values for df in train_dfs])

X_val = np.array([df[features].values for df in val_dfs])
y_val = np.array([df["Pick"].values for df in val_dfs])

# # Apply smoothening to your labels
# y_train = [np.clip(smooth_labels(y), 0, 1) for y in y_train]
# y_val = [np.clip(smooth_labels(y), 0, 1) for y in y_val]

# Apply smoothening to your labels
y_train = [smooth_labels(y) for y in y_train]
y_val = [smooth_labels(y) for y in y_val]


# Normalize data
X_train = [normalize_data(x, mean, std) for x in X_train]
X_val = [normalize_data(x, mean, std) for x in X_val]

# Ensure even length
X_train = [ensure_even_length(x) for x in X_train]
X_val = [ensure_even_length(x) for x in X_val]

# Compute max_length after ensuring even lengths
max_length = max(max(x.shape[0] for x in X_train), max(x.shape[0] for x in X_val))+2

# Pad to max_length
X_train = [pad_to_max_length(x, max_length) for x in X_train]
X_val = [pad_to_max_length(x, max_length) for x in X_val]

# Ensure even length for labels and then pad
y_train = [ensure_even_length(y.reshape(-1, 1)) for y in y_train]
y_val = [ensure_even_length(y.reshape(-1, 1)) for y in y_val]

y_train = [pad_to_max_length(y, max_length) for y in y_train]
y_val = [pad_to_max_length(y, max_length) for y in y_val]

C:\Users\nbive\AppData\Local\Temp\ipykernel_15568\2758920391.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array([df[features].values for df in train_dfs])
C:\Users\nbive\AppData\Local\Temp\ipykernel_15568\2758920391.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array([df["Pick"].values for df in train_dfs])
C:\Users\nbive\AppData\Local\Temp\ipykernel_15568\2758920391.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with 

In [42]:
def create_model(X_train, y_train, X_val, y_val, learning_rate, epochs):
    global_view = create_global_view()
    local_view = create_local_view()

    combined = layers.Multiply()([global_view.output, local_view.output])

    # Soft Attention
    attention_output = layers.Activation("tanh")(combined)

    # HYPERPARAMETER: Adjust the number of filters (1) and kernel size (1)
    output = layers.Conv1D(1, 1, activation="sigmoid")(attention_output)

    model = models.Model([global_view.input, local_view.input], output)
    optimizer = Adam(learning_rate=learning_rate)

    # HYPERPARAMETER: Adjust the optimizer ('adam') and its parameters
    model.compile(optimizer=optimizer, loss="binary_crossentropy")


    train_losses = []
    val_losses = []
    # We'll only use this training loop and remove the redundant one.
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Training
        epoch_train_losses = []
        for i in range(len(X_train)):
            X_train_well = X_train[i].reshape(1, X_train[i].shape[0], X_train[i].shape[1])
            y_train_well = y_train[i].reshape(1, y_train[i].shape[0], 1)
            loss = model.train_on_batch([X_train_well, X_train_well], y_train_well)  # Capturing the loss
            epoch_train_losses.append(loss)
        mean_train_loss = np.mean(epoch_train_losses)
        train_losses.append(np.mean(mean_train_loss))

        # Validation (optional)
        epoch_val_losses = []
        for i in range(len(X_val)):
            X_val_well = X_val[i].reshape(1, X_val[i].shape[0], X_val[i].shape[1])
            y_val_well = y_val[i].reshape(1, y_val[i].shape[0], 1)
            loss = model.test_on_batch([X_val_well, X_val_well], y_val_well)
            epoch_val_losses.append(loss)
        mean_val_loss = np.mean(epoch_val_losses)
        val_losses.append(mean_val_loss)
        print(f"Training Loss: {train_losses[-1]:.4f}, Validation Loss: {mean_val_loss:.4f}")



    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Curves")
    plt.legend()
    plt.show()

In [43]:
create_model(X_train, y_train,X_val, y_val, learning_rate = 0.00001,epochs=20)

Epoch 1/20
Training Loss: 0.5524, Validation Loss: 0.4215
Epoch 2/20
Training Loss: 0.2173, Validation Loss: -0.0920
Epoch 3/20
Training Loss: -0.2836, Validation Loss: -0.8316
Epoch 4/20
Training Loss: -1.0579, Validation Loss: -1.9379
Epoch 5/20
Training Loss: -2.1739, Validation Loss: -3.5574
Epoch 6/20
Training Loss: -3.7064, Validation Loss: -5.8951
Epoch 7/20
Training Loss: -5.6829, Validation Loss: -8.7592
Epoch 8/20


KeyboardInterrupt: 